In [1]:
import torch
import torchvision.models as models
import numpy as np
import json
import os
import cv2
import collections

#config
device = torch.device('cuda:0')
batch_size=32
lr=0.01
epochs=10

In [2]:
#input data
label=[]
img=[]
f1=open("./week10_dataset/0_annotation_train.txt",'r', encoding='UTF-8')
for line in f1:
    dic = json.loads(line)
    label.append(dic["annotation"][0]["name"])
    img.append('./week10_dataset'+dic["source"].split("image")[-1])

def train_data_generator(imgs, labels,classes, batch_size=32, img_size=(224,224)):
    batch_index = np.arange(0, len(imgs))
    img_out=[]
    label_out=[]
    while True:
        np.random.shuffle(batch_index)
        for i in batch_index:
            if os.path.exists(imgs[i]):
                img =cv2.imdecode(np.fromfile(imgs[i],dtype=np.uint8),1)
                label_out.append(classes.index(labels[i]))
                train_img= cv2.resize(img, (img_size[0], img_size[1]), interpolation=cv2.INTER_LINEAR)
                img_out.append(train_img)
                if len(img_out)>=batch_size:
                    img_out = np.array(img_out, dtype=np.float32)
                    img_out = img_out[:, :, :, ::-1]
                    img_out=torch.from_numpy(np.array(img_out))
                    img_out = img_out.permute(0, 3, 1, 2).float() / (255.0 / 2) - 1
                    label_out = np.array(label_out, dtype=np.int64)
                    label_out=torch.from_numpy(np.array(label_out))
                    label_out = label_out.long()
                    yield img_out, label_out
                    img_out, label_out=[], []
            else:
                print(imgs[i], 'not exist')

C = collections.Counter(label)
classes=list(C.keys())
data_generator=train_data_generator(img, label,classes, batch_size=batch_size, img_size=(224,224))


In [3]:
#model
model = models.resnet18(pretrained=True).to(device)
fc_features = model.fc.in_features
model.fc = torch.nn.Linear(fc_features, len(classes)).to(device)
optimizer = torch.optim.Adam([model.fc.weight,model.fc.bias], lr=lr)
criterion = torch.nn.CrossEntropyLoss()


In [4]:
#training
epoch_size=int(len(img)/batch_size)
for epoch in range(epochs):
    model.train()
    epoch_loss=0.0
    correct = 0
    total = 0
    for iter in range(1, epoch_size + 1):
        imgs, labels = next(data_generator)
        imgs = imgs.to(device)
        labels = labels.to(device)
        predicts = model(imgs)
        optimizer.zero_grad()
        loss = criterion(predicts, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        _, predicted = predicts.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        print( 'TRAIN:[epoch-%d , iter-%d]  Loss: %.3f | Acc: %.3f%% (%d/%d)' % (epoch, iter,epoch_loss / iter, 100. * correct / total, correct, total))



TRAIN:[epoch-0 , iter-1]  Loss: 2.746 | Acc: 0.000% (0/32)
TRAIN:[epoch-0 , iter-2]  Loss: 2.855 | Acc: 6.250% (4/64)
TRAIN:[epoch-0 , iter-3]  Loss: 3.211 | Acc: 8.333% (8/96)
TRAIN:[epoch-0 , iter-4]  Loss: 3.390 | Acc: 12.500% (16/128)
TRAIN:[epoch-1 , iter-1]  Loss: 2.902 | Acc: 34.375% (11/32)
TRAIN:[epoch-1 , iter-2]  Loss: 2.878 | Acc: 34.375% (22/64)
TRAIN:[epoch-1 , iter-3]  Loss: 2.480 | Acc: 40.625% (39/96)
TRAIN:[epoch-1 , iter-4]  Loss: 2.356 | Acc: 44.531% (57/128)
TRAIN:[epoch-2 , iter-1]  Loss: 1.827 | Acc: 81.250% (26/32)
TRAIN:[epoch-2 , iter-2]  Loss: 1.530 | Acc: 78.125% (50/64)
TRAIN:[epoch-2 , iter-3]  Loss: 1.543 | Acc: 70.833% (68/96)
TRAIN:[epoch-2 , iter-4]  Loss: 1.342 | Acc: 72.656% (93/128)
TRAIN:[epoch-3 , iter-1]  Loss: 1.087 | Acc: 53.125% (17/32)
TRAIN:[epoch-3 , iter-2]  Loss: 1.012 | Acc: 57.812% (37/64)
TRAIN:[epoch-3 , iter-3]  Loss: 1.005 | Acc: 61.458% (59/96)
TRAIN:[epoch-3 , iter-4]  Loss: 0.968 | Acc: 64.062% (82/128)
TRAIN:[epoch-4 , iter-1]  